# Scraping OECD's Glossary of Statistical Terms:  
### https://stats.oecd.org/glossary/

In [1]:
import urllib.request as urllib2 
from urllib.request import urlopen

import bs4
from bs4 import BeautifulSoup

import re
import pandas as pd

import requests

import numpy as np
from operator import itemgetter

In [2]:
base_url = "https://stats.oecd.org/glossary/detail.asp?ID={}"

sections = ['Definition','Definition:','Statistical Theme:','Context:',
            'Last updated on','Created on','Cross References:','Source Publication:','French Equivalent:','Hyperlink:','Glossary Output Segments:','Version Indicator:','Classification Indicator:','French Definition:']


num_articles = 8000
results=pd.DataFrame(index=range(num_articles))

for n in range(1,num_articles+1):
    if n % 100 ==0: print(n)
    scrape_url = base_url.format(n)
    results.loc[n,'ID'] = n
    results.loc[n,'URL'] = scrape_url
    res = requests.get(scrape_url)
    soup = bs4.BeautifulSoup(res.content,'lxml')
    
    links = []
    for link in soup.findAll('a'):
        links.append(link.get('href'))

    #print(links)
    str1 = " ".join(str(x) for x in links)
    p=re.findall(r'detail\.asp\?ID=\d+', str1)
    #print(p)
    p = ['https://stats.oecd.org/glossary/'+lnk for lnk in p]
    
    text=soup.get_text()
    text=re.sub(r'(\n)+',' ',text)
    text=re.sub(r'( )+',' ',text)
    text=text.replace('\xa0','')
    text=text.replace('\r',' ')
    text=re.sub(r'( )+',' ',text)
    text = text.replace('OECD Glossary of Statistical Terms -', "")
    text = text.replace('Glossary Home About Contact Us Downloadable Version Advanced Filter Web Service OECD Statistics', "")


    #print('ID = ',n,': >',text,'<')
    first_column = True ## first to be collected: Term
    matches = [(re.search(section,text).span()[0],re.search(section,text).span()[1],section) 
               for section in sections if not re.search(section,text) is None]
    matches.sort(key=itemgetter(0))
    #print('matches: ',matches,'\n')
    for k in range(len(matches)):
        match=matches[k] 
        s1,s2 = match[:2] ## start-end
        column = match[2] ## item from list 'sections'
        #print('checking: ',s1,s2,column)
        if k == len(matches)-1: ## last one, end of text to collect is end of 'text'
            s1_next = len(text)
        else:                   ## not last one, end of text to collect is start of next match  
            match_next=matches[k+1]
            s1_next = match_next[0]
        if first_column:
            results.loc[n,'Term'] = text[:s1].strip()
            first_column = False ## finished with Term - will be reset to True in next 'text'
        else:    
            results.loc[n,column] = text[s2:s1_next].strip()
        #print(p)
        results.loc[n,'URL:Cross References']=','.join(p)
        
        

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000


In [3]:
results.dropna(subset = ['Term'],inplace=True)
results.reset_index(drop=True,inplace=True)
results['ID'] = results['ID'].astype('int32')
results

,ID,URL,Term,URL:Cross References,French Equivalent:,Definition:,Cross References:,Statistical Theme:,Created on,Last updated on,Source Publication:,Context:,Hyperlink:,Glossary Output Segments:,Classification Indicator:,Version Indicator:,French Definition:
0,1,https://stats.oecd.org/glossary/detail.asp?ID=1,Abatement,https://stats.oecd.org/glossary/detail.asp?ID=...,Réduction,See Pollution abatement.,Pollution abatement,Environmental statistics,"Tuesday, September 25, 2001","Thursday, March 14, 2002",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,https://stats.oecd.org/glossary/detail.asp?ID=2,Absence from work due to illness,,NaN,Absence from work due to illness refers to the...,NaN,Health statistics,"Tuesday, September 25, 2001","Thursday, November 22, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
2,3,https://stats.oecd.org/glossary/detail.asp?ID=3,Activity restriction - free expectancy,,NaN,Functional limitation-free life expectancy is ...,NaN,Health statistics,"Tuesday, September 25, 2001","Wednesday, October 31, 2001",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
3,4,https://stats.oecd.org/glossary/detail.asp?ID=4,Acute care,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care is one in which the principal inten...,Acute care beds Acute care hospital staff rati...,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",OECD Health Data 2001: A Comparative Analysis ...,NaN,NaN,NaN,NaN,NaN,NaN
4,5,https://stats.oecd.org/glossary/detail.asp?ID=5,Acute care beds,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,Acute care beds are beds accommodating patient...,Acute care Long-term care beds in hospitals,Health statistics,"Tuesday, September 25, 2001","Thursday, April 25, 2013",2001 Data Collection on Education Systems: Def...,Acute care beds have alternatively been define...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6934,7353,https://stats.oecd.org/glossary/detail.asp?ID=...,CO2,https://stats.oecd.org/glossary/detail.asp?ID=284,NaN,NaN,Carbon dioxide (CO2),NaN,"Thursday, April 4, 2013","Thursday, April 4, 2013",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6935,7354,https://stats.oecd.org/glossary/detail.asp?ID=...,Carbon market,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,A popular (but misleading) term for a trading ...,Greenhouse gases,NaN,"Thursday, April 4, 2013","Thursday, April 4, 2013",United Nations Framework Convention on Climate...,NaN,http://unfccc.int/essential_background/glossar...,NaN,NaN,NaN,NaN
6936,7355,https://stats.oecd.org/glossary/detail.asp?ID=...,Classification structure,https://stats.oecd.org/glossary/detail.asp?ID=350,NaN,Refers to how the categories of a classificati...,Classification,NaN,"Tuesday, April 9, 2013","Tuesday, April 9, 2013","United Nations Statistics Division, n.d. UN Gl...",NaN,http://unstats.un.org/unsd/class/family/glossa...,NaN,NaN,NaN,NaN
6937,7356,https://stats.oecd.org/glossary/detail.asp?ID=...,United Nation Framework Convention on Climate ...,https://stats.oecd.org/glossary/detail.asp?ID=...,NaN,The United Nations Framework Convention on Cli...,United Nations Conference on Environment and D...,NaN,"Tuesday, April 9, 2013","Friday, April 26, 2013",United Nations Framework Convention on Climate...,"The other “Rio Conventions”, also negotiated a...",http://unfccc.int/2860.php,NaN,NaN,NaN,NaN


* Add column 'Cross_References_2' with cross-references separated by semicolons.

In [9]:
for i in range(len(results)):

    if not results.loc[i,'URL:Cross References'].strip()=='':
        links = results.loc[i,'URL:Cross References'].split(',')
        matches=[re.search(r'\d+$',el) for el in links]
        x = [int(el[m.span()[0]:m.span()[1]]) for (el,m) in zip(links,matches)]
        titles = [results.loc[np.where(results.ID==id)[0],'Term'].values[0] for id in x]
        #print(i,titles)
        results.loc[i,'Cross_References_2']=';'.join([t for t in titles if not t is np.nan])

In [10]:
results.to_excel('OECD_final_results_2.xlsx')